In [5]:
import os
import sys
import pickle
import time
import numpy as np
from tqdm.auto  import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import SubsetRandomSampler
from torch_scatter import scatter_mean

from torch_geometric.data import DataLoader
from torch_geometric.nn import (NNConv, GMMConv, GraphConv, Set2Set)
from torch_geometric.data import (InMemoryDataset, Data)
import torch_geometric.transforms as T
from torch_geometric.utils import normalized_cut
# import torch_spline_conv
from torch_geometric.nn import (SplineConv, graclus, max_pool, max_pool_x, global_mean_pool)
# from torch_geometric.nn import graclus,  max_pool, max_pool_x, global_mean_pool
# from torch_spline_conv import (SplineConv)
# import torch_spline_conv

# from torch_geometric.datasets import MNISTSuperpixels
import keras

import progressbar

Using TensorFlow backend.


In [118]:

X_train = np.load("skel_features/mnist/save/X_train.npy", allow_pickle=True)
X_test = np.load("skel_features/mnist/save/X_test.npy", allow_pickle=True)
y_train = np.load("skel_features/mnist/save/y_train.npy", allow_pickle=True)
y_test = np.load("skel_features/mnist/save/y_test.npy", allow_pickle=True)
skeleton_train = np.load("skel_features/mnist/save/skeleton_train.npy", allow_pickle=True)
skeleton_test = np.load("skel_features/mnist/save/skeleton_test.npy", allow_pickle=True)

# max_len = np.max([x.shape[0] for x in np.hstack([skeleton_train, skeleton_test])])

# for i in range(len(skeleton_train)):
#     tmp = np.zeros((max_len))
#     tmp[:len(skeleton_train[i])] = skeleton_train[i]
#     skeleton_train[i] = tmp
    
# for i in range(len(skeleton_test)):
#     tmp = np.zeros((max_len))
#     tmp[:len(skeleton_test[i])] = skeleton_test[i]
#     skeleton_test[i] = tmp
    
X_train = np.vstack(X_train).reshape(60000,28,28,1)
X_test = np.vstack(X_test).reshape(10000,28,28,1)
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)
# skeleton_train = np.vstack(skeleton_train)
# skeleton_test = np.vstack(skeleton_test)

# num_features = max_len

### we need to transform skeleton to edge_index

In [133]:
!pip install progressbar

  Created wheel for progressbar: filename=progressbar-2.5-cp37-none-any.whl size=12073 sha256=a7fdd012292f667077a0a5f675296b954d90a7e4fadb3f4d16ad675c99bb4257
  Stored in directory: /Users/aydar/Library/Caches/pip/wheels/c0/e9/6b/ea01090205e285175842339aa3b491adeb4015206cda272ff0
Successfully built progressbar


In [141]:
bar = progressbar.ProgressBar(skeleton_train.shape[0])
bar.start()

edge_index_train = []
for j, skeleton in enumerate(skeleton_train):
    skeleton = skeleton.reshape(-1,8)
    nodes_l = skeleton[:, 0:2]
    nodes_r = skeleton[:, 4:6]
    
    nodes = np.unique(np.vstack((nodes_l, nodes_r)), axis=0)
    node_ind = {}
    for i, node in enumerate(nodes):
        node_ind[str(node)] = i

    edge_index = [[],[]]
    for i in range(nodes_l.shape[0]):
        node0 = node_ind[str(nodes_l[i])]
        node1 = node_ind[str(nodes_r[i])]
        edge_index[0].append(node0)
        edge_index[1].append(node1)
    
    edge_index_train.append(edge_index)
    
    bar.update(j)
    
    

In [140]:
bar = progressbar.ProgressBar(skeleton_test.shape[0])
bar.start()

edge_index_test = []
for j, skeleton in enumerate(skeleton_test):
    skeleton = skeleton.reshape(-1,8)
    nodes_l = skeleton[:, 0:2]
    nodes_r = skeleton[:, 4:6]
    
    nodes = np.unique(np.vstack((nodes_l, nodes_r)), axis=0)
    node_ind = {}
    for i, node in enumerate(nodes):
        node_ind[str(node)] = i

    edge_index = [[],[]]
    for i in range(nodes_l.shape[0]):
        node0 = node_ind[str(nodes_l[i])]
        node1 = node_ind[str(nodes_r[i])]
        edge_index[0].append(node0)
        edge_index[1].append(node1)
    
    edge_index_test.append(edge_index)
    
    bar.update(j)

In [149]:
edge_index_test[0]

[[1,
  2,
  3,
  6,
  8,
  16,
  17,
  18,
  21,
  22,
  23,
  25,
  27,
  30,
  31,
  34,
  36,
  38,
  37,
  35,
  33,
  32,
  29,
  28,
  26,
  24,
  20,
  19,
  15,
  14,
  13,
  12,
  11,
  10,
  9,
  7,
  5,
  4,
  2],
 [0,
  0,
  1,
  3,
  6,
  8,
  16,
  17,
  18,
  21,
  22,
  23,
  25,
  27,
  30,
  31,
  34,
  36,
  38,
  37,
  35,
  33,
  32,
  29,
  28,
  26,
  24,
  20,
  19,
  15,
  14,
  13,
  12,
  11,
  10,
  9,
  7,
  5,
  4]]

In [148]:
Data(edge_index=edge_index_test[0])

Data(edge_index=[2])

In [127]:
# skeleton = skeleton_train.reshape(len(skeleton_train)//8, -1,8)
skeleton = skeleton_train[0].reshape(-1,8)
nodes_l = skeleton[:, 0:2]
nodes_r = skeleton[:, 4:6]
skeleton.shape

(35, 8)

In [129]:
nodes = np.unique(np.vstack((nodes_l, nodes_r)), axis=0)
node_ind = {}
for i, node in enumerate(nodes):
    node_ind[str(node)] = i
    
edge_index = [[],[]]
for i in range(nodes.shape[0]):
    node0 = node_ind[str(nodes_l[i])]
    node1 = node_ind[str(nodes_r[i])]
    edge_index[0].append(node0)
    edge_index[1].append(node1)
    

np.array(edge_index)

array([[ 1,  2,  4,  5,  5,  9, 12, 13, 15, 20, 22, 24, 27, 31, 33, 32,
        30, 29, 28, 26, 25, 23, 21, 19, 18, 17, 16, 14, 11, 10,  8,  7,
         6,  3],
       [ 0,  0,  1,  4,  5,  5,  9, 12, 13, 15, 20, 22, 24, 27, 31, 33,
        32, 30, 29, 28, 26, 25, 23, 21, 19, 18, 17, 16, 14, 11, 10,  8,
         7,  6]])

In [97]:
one

array([0., 0.])

In [98]:
(n == one).sum()

632

In [29]:
class MPNNNet(nn.Module):
    def __init__(self, num_features, aggr='mean', processing_steps=8, message_passing_steps=4, dim=64):
        super(MPNNNet, self).__init__()
        self.message_passing_steps = message_passing_steps
        
        self.lin0 = torch.nn.Linear(num_features, dim) # Change 2 to 1 for superpixels
        nn = torch.nn.Sequential(torch.nn.Linear(2, 128), torch.nn.ReLU(), torch.nn.Linear(128, dim * dim))
        self.conv = NNConv(dim, dim, nn, aggr=aggr, root_weight=False)
        self.gru = torch.nn.GRU(dim, dim)
        
        self.set2set = Set2Set(dim, processing_steps=processing_steps)
        self.lin1 = torch.nn.Linear(2 * dim, dim)
        self.lin2 = torch.nn.Linear(dim, 10)
        
    def forward(self, data):
        out = F.relu(self.lin0(data))
        h = out.unsqueeze(0)

        for i in range(self.message_passing_steps):
            m = F.relu(self.conv(out, data.edge_index, data.edge_attr))
            out, h = self.gru(m.unsqueeze(0), h)
            out = out.squeeze(0)

        out = self.set2set(out, data.batch)
        out = F.relu(self.lin1(out))
        return F.log_softmax(self.lin2(out), dim=1)

In [46]:
net = MPNNNet(num_features).double()

In [47]:
net.forward(torch.tensor(skeleton_train[0]))

AttributeError: 'Tensor' object has no attribute 'edge_index'

In [45]:
lin0 = torch.nn.Linear(num_features, 64)
lin0.double().forward(torch.tensor(skeleton_train[0]).double())


tensor([-1.3433,  0.1281, -0.7942, -2.8545, -3.9708,  4.4691,  5.3152,  1.5529,
        -1.3469,  0.3190,  4.4608, -4.8103, -5.8134,  2.0857, -0.6675,  5.8784,
         3.3209,  4.8704,  3.2758, -0.2552, -1.3161, -7.2281,  9.7473, -7.9327,
         3.6714,  3.1915,  2.3394, -1.1447, -2.8668,  1.5335, -0.9352,  6.6427,
        -2.9320,  0.0193, -1.8996,  6.2554,  2.4394,  4.3844,  0.2130, -8.6886,
        -4.8625,  1.1349, -4.8590,  0.4929,  9.6076,  4.8010,  4.6720,  1.7811,
        -3.8135, -0.0297,  2.8187, -5.1534,  3.5983,  3.8094, -5.2263,  0.1551,
        -1.7206,  2.4326,  5.9250, -1.3569,  5.2368,  7.3881, -1.6508,  1.7505],
       dtype=torch.float64, grad_fn=<AddBackward0>)

In [43]:
torch.tensor(skeleton_train[0]).double()

tensor([ 8.0000, 19.0000,  2.0000,  1.0000,  8.0000, 15.0000,  2.0000,  1.0000,
         8.0557, 14.5279,  2.0000,  1.0557,  8.0000, 15.0000,  2.0000,  1.0000,
         8.1716, 19.8284,  2.0000,  1.1716,  8.0000, 19.0000,  2.0000,  1.0000,
         8.7574, 21.2426,  2.0000,  1.7574,  8.1716, 19.8284,  2.0000,  1.1716,
         8.7574, 21.2426,  2.0000,  1.7574,  8.7574, 21.2426,  2.0000,  1.7574,
        10.0000, 21.5000,  2.0000,  1.5000,  8.7574, 21.2426,  2.0000,  1.7574,
        11.6459, 21.5000,  2.0000,  1.5000, 10.0000, 21.5000,  2.0000,  1.5000,
        12.0000, 21.4164,  2.0000,  1.4164, 11.6459, 21.5000,  2.0000,  1.5000,
        12.6000, 21.2000,  2.0000,  1.3416, 12.0000, 21.4164,  2.0000,  1.4164,
        15.2053, 19.8974,  2.0000,  1.3416, 12.6000, 21.2000,  2.0000,  1.3416,
        16.4868, 18.9737,  2.0000,  1.0886, 15.2053, 19.8974,  2.0000,  1.3416,
        16.7500, 18.7500,  2.0000,  1.0607, 16.4868, 18.9737,  2.0000,  1.0886,
        19.7500, 15.7500,  2.0000,  1.06